In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
# cfg = Config.fromfile('trash/trash/swin.py')
cfg = Config.fromfile('trash/trash/swin_retrain.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_data4.json'
# cfg.data.train.pipeline[2]['img_scale'] = (768, 768)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'valid_data4.json'
# cfg.data.val.pipeline[1]['img_scale'] = (768, 768)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (768, 768)

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 2

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/swint_fold4_retrain'
cfg.checkpoint_config = dict(max_keep_ckpts=10,interval=1)
# cfg.load_from = "/opt/ml/code/mmdetection_trash/savemodel/cascade_mask_rcnn_swin_base_patch4_window7.pth"
cfg.load_from = "/opt/ml/code/mmdetection_trash/work_dirs/swint_fold4/epoch_39.pth"

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [3]:
cfg

Config (path: trash/trash/swin_retrain.py): {'model': {'type': 'CascadeRCNN', 'pretrained': None, 'backbone': {'type': 'SwinTransformer', 'embed_dim': 128, 'depths': [2, 2, 18, 2], 'num_heads': [4, 8, 16, 32], 'window_size': 7, 'mlp_ratio': 4.0, 'qkv_bias': True, 'qk_scale': None, 'drop_rate': 0.0, 'attn_drop_rate': 0.0, 'drop_path_rate': 0.3, 'ape': False, 'patch_norm': True, 'out_indices': (0, 1, 2, 3), 'use_checkpoint': False}, 'neck': {'type': 'FPN', 'in_channels': [128, 256, 512, 1024], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'SmoothL1Loss', 'beta': 0.1111111111111111, 

In [4]:
model = build_detector(cfg.model)

In [5]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.11s)
creating index...
index created!


In [6]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-20 08:00:09,806 - mmdet - INFO - load checkpoint from /opt/ml/code/mmdetection_trash/work_dirs/swint_fold4/epoch_39.pth
2021-05-20 08:00:09,808 - mmdet - INFO - Use load_from_local loader


Done (t=1.48s)
creating index...
index created!


2021-05-20 08:00:11,169 - mmdet - INFO - Start running, host: root@0ff209eaebce, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/swint_fold4_retrain
2021-05-20 08:00:11,171 - mmdet - INFO - workflow: [('train', 1)], max: 45 epochs
2021-05-20 08:01:32,584 - mmdet - INFO - Epoch [1][50/1311]	lr: 9.890e-07, eta: 1 day, 2:39:31, time: 1.628, data_time: 0.080, memory: 13486, loss_rpn_cls: 0.2600, loss_rpn_bbox: 0.0273, s0.loss_cls: 1.8160, s0.acc: 94.4219, s0.loss_bbox: 0.2197, s0.loss_mask: 3.5410, s1.loss_cls: 0.1470, s1.acc: 94.6300, s1.loss_bbox: 0.0932, s1.loss_mask: 2.2047, s2.loss_cls: 0.0844, s2.acc: 94.0787, s2.loss_bbox: 0.0367, s2.loss_mask: 1.1914, loss: 9.6216, grad_norm: 56.4201
2021-05-20 08:02:46,154 - mmdet - INFO - Epoch [1][100/1311]	lr: 1.988e-06, eta: 1 day, 1:21:12, time: 1.471, data_time: 0.028, memory: 13486, loss_rpn_cls: 0.2865, loss_rpn_bbox: 0.0367, s0.loss_cls: 1.6481, s0.acc: 92.4648, s0.loss_bbox: 0.2964, s0.loss_mask: 2.7030, s1.loss_cls: 0.1845, s1.acc: 9

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 650/650, 0.7 task/s, elapsed: 939s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.61s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.11s).
Accumulating evaluation results...


2021-05-20 08:47:44,338 - mmdet - INFO - Epoch(val) [1][1311]	bbox_mAP: 0.3430, bbox_mAP_50: 0.4570, bbox_mAP_75: 0.3460, bbox_mAP_s: 0.0760, bbox_mAP_m: 0.3060, bbox_mAP_l: 0.5660, bbox_mAP_copypaste: 0.343 0.457 0.346 0.076 0.306 0.566


DONE (t=0.98s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.343
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.457
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.346
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.076
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.306
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.566
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.460
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.460
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.460
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.164
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.431
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.704


2021-05-20 08:50:06,544 - mmdet - INFO - Epoch [2][50/1311]	lr: 1.000e-05, eta: 23:50:23, time: 2.844, data_time: 0.081, memory: 13510, loss_rpn_cls: 0.0398, loss_rpn_bbox: 0.0233, s0.loss_cls: 0.1845, s0.acc: 93.3633, s0.loss_bbox: 0.2317, s0.loss_mask: 0.3750, s1.loss_cls: 0.0249, s1.acc: 93.3326, s1.loss_bbox: 0.0970, s1.loss_mask: 0.1961, s2.loss_cls: 0.0518, s2.acc: 93.3906, s2.loss_bbox: 0.0365, s2.loss_mask: 0.1023, loss: 1.3630, grad_norm: 7.3642
2021-05-20 08:52:23,086 - mmdet - INFO - Epoch [2][100/1311]	lr: 1.000e-05, eta: 1 day, 0:31:22, time: 2.731, data_time: 0.028, memory: 13510, loss_rpn_cls: 0.0468, loss_rpn_bbox: 0.0275, s0.loss_cls: 0.2017, s0.acc: 92.2148, s0.loss_bbox: 0.2564, s0.loss_mask: 0.3453, s1.loss_cls: 0.0267, s1.acc: 92.1410, s1.loss_bbox: 0.1085, s1.loss_mask: 0.1848, s2.loss_cls: 0.0584, s2.acc: 92.5919, s2.loss_bbox: 0.0413, s2.loss_mask: 0.0943, loss: 1.3917, grad_norm: 7.9661
2021-05-20 08:55:34,370 - mmdet - INFO - Epoch [2][150/1311]	lr: 1.000e-05,

KeyboardInterrupt: 